In [1]:
import pandas as pd
import numpy as np
import os
import requests
from io import BytesIO
import json
import yadisk
from datetime import datetime, date, timedelta
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import config
from yandex_disk_func import *
from parse_functions import *
from google_connector import *

# # указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\35_mediaplan_parser'
# забираем токен для подключения к гугл
service_key = config.service
google_sheet_link = 'https://docs.google.com/spreadsheets/d/11hF_txYEbKrQImRQ0x3Fia3PyJvDAqtpGI25idV7oa4/edit?usp=sharing'
gmail = config.gmail
sheet_name = 'prog'

# обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета
public_key = config.public_key 

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# создаем словарь, где сохраним каждый продукт в отдельный датаФрейм со своим ключом
main_dict = {}

In [4]:
get_data_from_ya_folder(yandex_folders, main_dict, flag='prog')                     


/01_yandex
/02_prog
beeline_plan_игроник_ингосстрах_кис_лето_banners_020724
firstdata_игроник_ингосстрах_срм_media_28.08.2024
mediaplan без ск
firstdata_игроник_национальная лотерея_принцесса нури_срм_media_27.08.2024 (1)
mediaplan 1000к
mediaplan 800к
mediaplan 600к
hybrid_бронь_игроник_национальная лотерея_день рождения нл_klientskiy_plan_20240920 (1)
медиаплан
mp_mobidriven_игроник_ингосстрах_ хореографы_24.09 (1)
мп_4 дня
мп_4 неделе
mp_mobidriven_игроник_ингосстрах_инго экосистема_05.09
мп
mp_mobidriven_игроник_национальная лотерия_27.08 (1)
мп
mp_roxot_игроник_psb_brand-booster_sep-dec24
псб bb banner
псб bb olv
plan_игроник_рсхб_beeline_свой вклад_banners_160724 (1)
plan_ингосстрах_ beeline_хореографы_banners_240924 (3)
beeline_plan_igronik_бриф по тендерам_101024
игроник_дом рф_smart tv_hybrid_klientskiy_plan_20240725
медиаплан (ск 40%)
медиаплан (ск 50%)
игроник_псб имидж спецпроект_klientskiy_plan_hybrid_vox_20240830
медиаплан розница
медиаплан бизнес
/03_Полина


In [5]:
df = pd.concat(main_dict, ignore_index=True)
df = df.fillna('')


int_columns = ['reach', 'impressions', 'clicks']
df[int_columns] = df[int_columns].apply(lambda x: x.astype('int'))

In [6]:
float_columns = ['unit price', 'budget_without_nds', 'budget_nds', 'vtr, %']
df[float_columns] = df[float_columns].apply(lambda x: x.astype('float')).round(2)

In [7]:
df

,supplier,report_name,sheet_name,brand,period,source,site/ssp,placement,targeting,geo,soc_dem,ad copy format,rotation type,unit price,frequency,reach,impressions,clicks,budget_without_nds,budget_nds,views,"vtr, %"
0,beeline,beeline_plan_игроник_ингосстрах_кис_лето_banners_020724,Plan_Media,Ингосстрах_КИС_Лето,15.07-14.08.24,beeline AI,DSP,Desktop + mobile \n(+in-app),"ГЕО: РФ \nЦА: Все 18+\nПользователи, кто заходил на сайты/установлено приложение конкурентов в категории страхования (Росгосстрах, АльфаСтрахование, Тинькофф Страхование, СберСтрахование) и в банковской категории (Сбер, Альфа, Тинькофф, Россгосстрах, Согаз и т.д.)",рф,все 18+,Banners,CPM,88.0,3,7666666,23000000,29900,2024000.00,2428800.00,0,0.0
1,beeline,beeline_plan_игроник_ингосстрах_кис_лето_banners_020724,Plan_Media,Ингосстрах_КИС_Лето,15.07-14.08.24,beeline AI,DSP,Desktop + mobile \n(+in-app),ГЕО: РФ\nЦА: Все 18+\nКлиенты Сбера по транзакционным смс и установленному приложению,рф,все 18+,Banners,CPM,88.0,3,10000000,30000000,39000,2640000.00,3168000.00,0,0.0
2,firstdata,firstdata_игроник_ингосстрах_срм_media_28.08.2024,mediaplan без ск,Ингосстрах,3 months,Платформа First Data,,,"ГЕО: РФ\nЦА: МЖ. Доход ВС. \nПокупатели полисов КАСКО за 10-12 месяцев до старта рк. Владельцы авто на основании данных о покупках: автозапчастей, посещений автоателье, автосервисов, оплаты штрафов за эвакуацию.\nПокупатели услуг дизайнера интерьера, строительных компаний; товаров для строительства и ремонта: окна, двери, плитка, сантехника, напольные покрытия, краски, обои тд. Покупатели товаров для загородного дома: камины, барбекю, мангалы, газон, беседки, мебель уличная, газонокосилки и тд. \nИсточники: ОФД\n",рф,мж. доход вс.,Видео до 20сек.,CPM,175.0,5,3110000,15550000,31100,2721250.00,3265500.00,4665000,0.3
3,firstdata,firstdata_игроник_ингосстрах_срм_media_28.08.2024,mediaplan без ск,Ингосстрах,3 months,Платформа First Data,,,"ГЕО: РФ\nЦА: МЖ. Доход ВС. \nПокупатели полисов КАСКО за 10-12 месяцев до старта рк. Владельцы авто на основании данных о покупках: автозапчастей, посещений автоателье, автосервисов, оплаты штрафов за эвакуацию.\nПокупатели услуг дизайнера интерьера, строительных компаний; товаров для строительства и ремонта: окна, двери, плитка, сантехника, напольные покрытия, краски, обои тд. Покупатели товаров для загородного дома: камины, барбекю, мангалы, газон, беседки, мебель уличная, газонокосилки и тд. \nИсточники: ОФД\n",рф,мж. доход вс.,Баннеры / Универсальные Баннеры,CPM,130.0,5,4000000,20000000,40000,2600000.00,3120000.00,0.0,0.0
4,firstdata,firstdata_игроник_национальная лотерея_принцесса нури_срм_media_27.08.2024 (1),mediaplan 1000к,Национальная лотерея // Принцесса Нури,6 months,Платформа First Data,,,"ГЕО: РФ\nЦА: Ж, 35-45. Доход В, ВС. Покупатели чая и чайной продукции: Принцесса Нури, Азерчай, Ява, Riston, Майский, Ахмад, Dilmah, Ява, Милфорд, Липтон и др. Покупатели кондитерских изделий: безе, варенье, джем, повидло, грильяж, желе, зефир, пастила, конфеты, ирис, халва, помадка, муссы, шоколад, торты и др\nИсточники: ОФД",рф,"ж, 35-45. доход в, вс.",Баннеры / Универсальные Баннеры,CPM,240.0,12,2083333,25000000,50000,6000000.00,7200000.00,0,0.0
5,firstdata,firstdata_игроник_национальная лотерея_принцесса нури_срм_media_27.08.2024 (1),mediaplan 800к,Национальная лотерея // Принцесса Нури,6 months,Платформа First Data,,,"ГЕО: РФ\nЦА: Ж, 35-45. Доход В, ВС. Покупатели чая и чайной продукции: Принцесса Нури, Азерчай, Ява, Riston, Майский, Ахмад, Dilmah, Ява, Милфорд, Липтон и др. Покупатели кондитерских изделий: безе, варенье, джем, повидло, грильяж, желе, зефир, пастила, конфеты, ирис, халва, помадка, муссы, шоколад, торты и др\nИсточники: ОФД",рф,"ж, 35-45. доход в, вс.",Баннеры / Универсальные Баннеры,CPM,240.0,12,1666666,20000000,40000,4800000.00,5760000.00,0,0.0
6,firstdata,firstdata_игроник_национальная лотерея_принцесса нури_срм_media_27.08.2024 (1),mediaplan 600к,Национальная лотерея // Принцесса Нури,6 months,Платформа First Data,,,"ГЕО: РФ\nЦА: Ж,

In [ ]:
test = df[df['report_name']=='beeline_plan_igronik_бриф по тендерам_101024']

In [ ]:
test = df[df['sheet_name']=='mediaplan 1000к']

In [ ]:
test

In [ ]:
test.iloc[1]

In [ ]:
test[['reach', 'impressions', 'clicks', 'budget_without_nds']].sum()

In [ ]:
df

In [ ]:
# создаем подключение к Гуглу
client = create_connection(service_key)

In [ ]:
# прописываем путь и подключаемсяк нужному листу Гугл докса
sh = client.open_by_url(google_sheet_link)
sh.share(gmail, perm_type='user', role='writer')
google_sheet = sh.worksheet(sheet_name)

In [ ]:
# выгружем наш датаФрейм на лист
google_sheet.clear()
export_dataframe_to_google_sheet(google_sheet, final_report)

In [ ]:
main_dict['beeline_plan_игроник_ингосстрах_кис_лето_banners_020724']

In [ ]:
main_dict['plan_игроник_рсхб_beeline_свой вклад_banners_160724 (1)'].iloc[0]

In [ ]:
main_dict['plan_ингосстрах_ beeline_хореографы_banners_240924 (3)']

In [ ]:
main_dict['beeline_plan_igronik_бриф по тендерам_101024']